# Setup

In [25]:
# Impoting the Required Libraries 
import pandas as pd
import numpy as np

# DF Confirmed Cases

In [26]:
path = r'C:\Users\H241677\Desktop\ML\corona\data\Novel data set\covid_19_confirmed.csv'
df_cnf = pd.read_csv(path)
df_cnf.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,47,48,50,50,50,53,59,70,75,82
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,360,420,461,502,511,581,639,639,701,773
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,117,130,138,150,150,160,178,178,200,212
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,50,83,93,99,117,129,149,149,197,238


In [27]:
df_cnf.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
437,NaN,Saint Lucia,13.9094,-60.9789,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
438,NaN,Saint Vincent and the Grenadines,12.9843,-61.2872,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
439,NaN,Suriname,3.9193,-56.0278,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
440,"Virgin Islands, U.S.",US,18.3358,-64.8963,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
441,NaN,occupied Palestinian territory,31.9522,35.2332,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Data Cleaning 

In [28]:
df_cnf.drop(['Province/State'] , axis=1, inplace=True)

In [29]:
df_cnf.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20
0,Thailand,15.0000,101.0000,2,3,5,7,8,8,14,...,47,48,50,50,50,53,59,70,75,82
1,Japan,36.0000,138.0000,2,1,2,2,4,4,7,...,360,420,461,502,511,581,639,639,701,773
2,Singapore,1.2833,103.8333,0,1,3,3,4,5,7,...,117,130,138,150,150,160,178,178,200,212
3,Nepal,28.1667,84.2500,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,Malaysia,2.5000,112.5000,0,0,0,3,4,4,4,...,50,83,93,99,117,129,149,149,197,238


In [30]:
df_cnf['Total'] = df_cnf.loc[:,'1/22/20':'3/14/20'].sum(axis=1) 

In [31]:
df_cnf.head()

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,Total
0,Thailand,15.0000,101.0000,2,3,5,7,8,8,14,...,48,50,50,50,53,59,70,75,82,1781
1,Japan,36.0000,138.0000,2,1,2,2,4,4,7,...,420,461,502,511,581,639,639,701,773,9153
2,Singapore,1.2833,103.8333,0,1,3,3,4,5,7,...,130,138,150,150,160,178,178,200,212,3895
3,Nepal,28.1667,84.2500,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,50
4,Malaysia,2.5000,112.5000,0,0,0,3,4,4,4,...,83,93,99,117,129,149,149,197,238,2015


In [33]:
a = df_cnf.groupby('Country/Region', as_index=False).mean()
a.tail(10)

,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,Total
133,Tunisia,34.000000,9.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,1.000000,1.000000,2.000000,2.000000,5.000000,7.000000,7.000000,16.000000,18.000000,61.000000
134,Turkey,38.963700,35.243300,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,5.000000,5.000000,12.000000
135,US,38.689265,-93.888677,0.004065,0.004065,0.00813,0.00813,0.020325,0.020325,0.020325,...,1.065041,1.634146,2.105691,2.369919,3.898374,5.207317,6.760163,8.857724,11.085366,48.333333
136,Ukraine,48.379400,31.165600,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,3.000000,16.000000
137,United Arab Emirates,24.000000,54.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,29.000000,45.000000,45.000000,45.000000,74.000000,74.000000,85.000000,85.000000,85.000000,972.000000
138,United Kingdom,46.963733,-3.718000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,54.666667,69.000000,91.333333,107.333333,128.000000,153.000000,153.000000,267.000000,381.000000,1595.333333
139,Uruguay,-32.522800,-55.765800,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000
140,Venezuela,6.423800,-66.589700,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000
141,Vietnam,16.000000,108.000000,0.000000,2.000000,2.00000,2.00000,2.000000,2.000000,2.000000,...,16.000000,18.000000,30.000000,30.000000,31.000000,38.000000,39.000000,47.000000,53.000000,798.000000
142,occupied Palestinian territory,31.952200,35.233200,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
